<a href="https://colab.research.google.com/github/jus-tinian/vix_calc/blob/master/SparkTD_SPXquotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create Working Env

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
!pip install -q findspark
!pip install -q pyspark

## Imports

In [0]:
import requests
import datetime

import pyspark
import pandas as pd

from pyspark.sql import SparkSession

from pyspark.sql.functions import udf
from pyspark.sql.types import TimestampType, DateType, StructType, StructField, StringType
from pyspark.sql.types import FloatType, LongType, IntegerType
from pyspark.sql import DataFrame

from functools import reduce

## Create Spark Session

In [0]:
APP_NAME = "VIXcalc"
SPARK_URL = "local[*]"

In [0]:
spark = SparkSession.builder.master(SPARK_URL).appName(APP_NAME).getOrCreate()

In [8]:
spark

## Make Dates

In [0]:
today = datetime.datetime.today().date()

In [0]:
farDate = today + datetime.timedelta(days=37)

In [0]:
nearDate = today + datetime.timedelta(days=23)

## Get API response

In [0]:
# your key here
KEY = 'your key here'

In [0]:
url = f"https://api.tdameritrade.com/v1/marketdata/chains?apikey={KEY}&symbol=%24SPX.X&includeQuotes=TRUE&strategy=SINGLE&range=ALL&fromDate={str(nearDate)}&toDate={str(farDate)}"

In [0]:
req = requests.get(url)

In [0]:
reqJSON = req.json()

## Set Keys to access reqJSON

In [0]:
putKey = 'putExpDateMap'

In [0]:
callKey = 'callExpDateMap'

In [0]:
dateKeys = [i for i in list(req.json()[putKey].keys()) if datetime.datetime.strptime(i[:-3], '%Y-%m-%d').weekday() == 4]

In [0]:
nearKey = min(dateKeys)
farKey = max(dateKeys)

## Set Interest Rate

In [0]:
RATE = reqJSON['interestRate']

## UDFs

In [0]:
convertEpochToDate = udf(lambda msEpoch: datetime.datetime.fromtimestamp(msEpoch/1000).date(), DateType())

In [0]:
convertEpochToTimeStamp = udf(lambda msEpoch: datetime.datetime.fromtimestamp(msEpoch/1000), TimestampType())

## Prep Pipe NO TOUCH

In [0]:
from pyspark.sql import Row

In [0]:
def from_json(jsonObj, sideKey, termKey):

  # make list of strikes
  strikesList = list(jsonObj[sideKey][termKey].keys())
  keep = ['strikePrice', 'symbol', 'putCall', 'bid', 'ask', 'expirationDate', 'inTheMoney']

  source = [Row(**{k:v for k,v in reqJSON[sideKey][termKey][strike][0].items() if k in keep}) for strike in strikesList]

  # similar to for loop to append all DFs into single DF
  source_df1 = spark.createDataFrame(source)
  
  # drop non zero bids
  source_df2 = source_df1.where('bid != 0')

  # for Friday when there is a weekly & standard expiry, select standard
  nonWeeklyCount = source_df2.filter(~source_df2.symbol.contains('W')).count()
  if nonWeeklyCount != 0:
    source_df2 = source_df2.filter(~source_df2.symbol.contains('W'))

  # convert msEpoch (expirationDate) to datetime
  source_df3 = source_df2.withColumn('expiration', convertEpochToDate('expirationDate'))
      
  return source_df3

# Near Puts

In [0]:
nearPuts = from_json(reqJSON, putKey, nearKey)

In [26]:
nearPuts.show()

+----+----+--------------+----------+-------+-----------+----------------+----------+
| ask| bid|expirationDate|inTheMoney|putCall|strikePrice|          symbol|expiration|
+----+----+--------------+----------+-------+-----------+----------------+----------+
|0.15|0.05| 1591992000000|     false|    PUT|     1300.0|SPXW_061220P1300|2020-06-12|
|0.15| 0.1| 1591992000000|     false|    PUT|     1350.0|SPXW_061220P1350|2020-06-12|
|0.15| 0.1| 1591992000000|     false|    PUT|     1400.0|SPXW_061220P1400|2020-06-12|
| 0.2| 0.1| 1591992000000|     false|    PUT|     1450.0|SPXW_061220P1450|2020-06-12|
| 0.2| 0.1| 1591992000000|     false|    PUT|     1500.0|SPXW_061220P1500|2020-06-12|
| 0.2|0.15| 1591992000000|     false|    PUT|     1550.0|SPXW_061220P1550|2020-06-12|
|0.25|0.15| 1591992000000|     false|    PUT|     1600.0|SPXW_061220P1600|2020-06-12|
|0.25| 0.2| 1591992000000|     false|    PUT|     1650.0|SPXW_061220P1650|2020-06-12|
| 0.3| 0.2| 1591992000000|     false|    PUT|     1700

## Far Puts

In [0]:
farPuts = from_json(reqJSON, putKey, farKey)

In [28]:
farPuts.show()

+----+----+--------------+----------+-------+-----------+---------------+----------+
| ask| bid|expirationDate|inTheMoney|putCall|strikePrice|         symbol|expiration|
+----+----+--------------+----------+-------+-----------+---------------+----------+
| 0.1|0.05| 1592596800000|     false|    PUT|      900.0| SPX_061920P900|2020-06-19|
| 0.1|0.05| 1592596800000|     false|    PUT|     1000.0|SPX_061920P1000|2020-06-19|
| 0.1|0.05| 1592596800000|     false|    PUT|     1100.0|SPX_061920P1100|2020-06-19|
| 0.1|0.05| 1592596800000|     false|    PUT|     1150.0|SPX_061920P1150|2020-06-19|
| 0.1|0.05| 1592596800000|     false|    PUT|     1200.0|SPX_061920P1200|2020-06-19|
|0.15|0.05| 1592596800000|     false|    PUT|     1225.0|SPX_061920P1225|2020-06-19|
|0.15|0.05| 1592596800000|     false|    PUT|     1250.0|SPX_061920P1250|2020-06-19|
| 0.2|0.05| 1592596800000|     false|    PUT|     1275.0|SPX_061920P1275|2020-06-19|
| 1.6| 1.5| 1592596800000|     false|    PUT|     2000.0|SPX_0619

# Near Calls

In [0]:
nearCalls = from_json(reqJSON, callKey, nearKey)

In [30]:
nearCalls.show()

+------+------+--------------+----------+-------+-----------+----------------+----------+
|   ask|   bid|expirationDate|inTheMoney|putCall|strikePrice|          symbol|expiration|
+------+------+--------------+----------+-------+-----------+----------------+----------+
|2455.7|2449.2| 1591992000000|      true|   CALL|      500.0| SPXW_061220C500|2020-06-12|
|2355.7|2349.3| 1591992000000|      true|   CALL|      600.0| SPXW_061220C600|2020-06-12|
|2255.4|2249.3| 1591992000000|      true|   CALL|      700.0| SPXW_061220C700|2020-06-12|
|2155.8|2149.4| 1591992000000|      true|   CALL|      800.0| SPXW_061220C800|2020-06-12|
|2055.4|2049.4| 1591992000000|      true|   CALL|      900.0| SPXW_061220C900|2020-06-12|
|1955.9|1949.4| 1591992000000|      true|   CALL|     1000.0|SPXW_061220C1000|2020-06-12|
|1905.9|1899.5| 1591992000000|      true|   CALL|     1050.0|SPXW_061220C1050|2020-06-12|
|1855.9|1849.5| 1591992000000|      true|   CALL|     1100.0|SPXW_061220C1100|2020-06-12|
|1805.6|17

## Far Calls

In [0]:
farCalls = from_json(reqJSON, callKey, farKey)

In [32]:
farCalls.show()

+------+------+--------------+----------+-------+-----------+---------------+----------+
|   ask|   bid|expirationDate|inTheMoney|putCall|strikePrice|         symbol|expiration|
+------+------+--------------+----------+-------+-----------+---------------+----------+
|2853.8|2850.9| 1592596800000|      true|   CALL|      100.0| SPX_061920C100|2020-06-19|
|2753.8|2750.9| 1592596800000|      true|   CALL|      200.0| SPX_061920C200|2020-06-19|
|2653.9|2651.0| 1592596800000|      true|   CALL|      300.0| SPX_061920C300|2020-06-19|
|2553.9|2551.0| 1592596800000|      true|   CALL|      400.0| SPX_061920C400|2020-06-19|
|2453.9|2451.1| 1592596800000|      true|   CALL|      500.0| SPX_061920C500|2020-06-19|
|2353.9|2351.0| 1592596800000|      true|   CALL|      600.0| SPX_061920C600|2020-06-19|
|2254.0|2251.1| 1592596800000|      true|   CALL|      700.0| SPX_061920C700|2020-06-19|
|1953.9|1951.3| 1592596800000|      true|   CALL|     1000.0|SPX_061920C1000|2020-06-19|
|1854.2|1851.3| 15925